RAG, RAG, RAG..... 

One of most popular and used technique related to LLMs, used to enhance the performance of the LLM for customized task by giving it a Knowledge Base.



Everyone is making useful Chat Bots for the Customized data but industry is using some advance RAG Techniques to get the accurate results from their RAG Systems.



Some of those Advance RAG Techniques are:

1. 𝐐𝐮𝐞𝐫𝐲 𝐄𝐱𝐩𝐚𝐧𝐬𝐢𝐨𝐧 𝐔𝐬𝐢𝐧𝐠 𝐭𝐡𝐞 𝐇𝐲𝐩𝐨𝐭𝐡𝐞𝐭𝐢𝐜𝐚𝐥 𝐀𝐧𝐬𝐰𝐞𝐫 𝐟𝐫𝐨𝐦 𝐋𝐋𝐌:

Get a hypothetical answer from LLM for query, combine both of them to get an accurate and closely related answers/documents from the vector database.

2. 𝐐𝐮𝐞𝐫𝐲 𝐄𝐱𝐩𝐚𝐧𝐬𝐢𝐨𝐧 𝐔𝐬𝐢𝐧𝐠 𝐭𝐡𝐞 𝐑𝐞𝐥𝐚𝐭𝐞𝐝 𝐐𝐮𝐞𝐫𝐢𝐞𝐬:

Generate the queries related to original query using LLM and then we can use these queries together to get more appropriate related documents from our vector database.

3. 𝐂𝐫𝐨𝐬𝐬 𝐄𝐧𝐜𝐨𝐝𝐞𝐫 𝐑𝐞-𝐑𝐚𝐧𝐤𝐢𝐧𝐠:

Using this technique we can score the retrieval of the documents according to the query that we have set. Basically Every document will have a score, that will show how much related it is to our query.


In [1]:
!pip install langchain chromadb pypdf sentence-transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importing Libraries and Functions:

In [2]:
import chromadb
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from pypdf import PdfReader
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.document_loaders import TextLoader
from langchain.llms import CTransformers
from torch import cuda, bfloat16
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

2024-03-04 04:28:58.312519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 04:28:58.312619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 04:28:58.488237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# llama2 Model and Tokenizer:

In [3]:
model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


# Function for Loading the Doucument:

We are removing whitespaces and other stuff becuase, we will not be able to ingest none type objects in vector database so we have to process the pdf before ingesting it in the vector database.

This function will return us the text from our Document after removing extra white spaces and empty strings/pages to prevent errors.:

In [5]:
def load_doc(path):
    reader = PdfReader(path)
    #We will iterate over each page 'p', extract the text and removing the whitespaces before and after sentences.
    pdf_texts = [p.extract_text().strip() for p in reader.pages]

    # Filter the empty strings-->pdf_texts will have only those pages that have the text
    pdf_texts = [text for text in pdf_texts if text]

    return pdf_texts

# Function for Dividing the Documents in Chunks:
We do chunking to ensure that our document is easily processed by LLM and chunks will be easily commited in the Vector Database.

This Function will return us the list of Chunks that are result of our Document Splitting.

Character Splitter is not enogh becuase the embedding model that we are using called SentenceTransformer has limited context window width and exactly uses 256 characters. That is the maximum context window width which is basically maximum legth of text used to understand the that text.

In [6]:
def chunks(path_of_doc):

    #we will use the path of doc to pass it to load_doc
    pdf_texts= load_doc(path_of_doc)
    character_splitter = RecursiveCharacterTextSplitter(
    #RecursiveCharacterTextSplitter will split the Document into chunks firstly when it will find the double line
        #After that if the splitted chunks are greater than size of 1000 which is our chunk size they will get split on single line
            #Even if Chunks have larger size than 1000 they will get split on ". "
        separators=["\n\n", "\n", ". ", " ", ""],
        #every chunk will have 2000 characters
        chunk_size=2000,
        chunk_overlap=20
    )
    #further splitting according to embedding model we are using
    character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))
    #so lets split it according to our embedding model
                                                                            #we want every token to have 256 characters
    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)
    token_split_texts = []
    for text in character_split_texts:
    #splitting the text and storing in list
        token_split_texts += token_splitter.split_text(text)

    return token_split_texts

# Function for Embeddings:

We will convert our textual data/chunks to numbers or vectors to store them in vector database.
This Function will return us a method to Embedd our textual chunks to numbers.

In [7]:
def embedding_function():
    embedding_function = SentenceTransformerEmbeddingFunction()
    return embedding_function

# Function for Doc/Chunks Ingestion in the Vector Database:
This function will return us the collection of Vector database that will have our chunks vector stored according to semantic meaning and we will able to query this database/collection to get answer from our document that we ingested.

In [8]:
def doc_ingestion_in_vecdb(path_of_doc):
    #we will pass the path of doc to chunks that will further pass it to doc_load for loading it
    token_split_texts= chunks(path_of_doc)
    embedding_func= embedding_function()

    #making chromadb client object good for testing only not for production purpose
    chroma_client = chromadb.Client()

    #making the collection of chroma database
    chroma_collection = chroma_client.create_collection("Doc-Collection", embedding_function=embedding_func)

    #ids of each chunk
    ids = [str(i) for i in range(len(token_split_texts))]

    chroma_collection.add(ids=ids, documents=token_split_texts)
    return chroma_collection

# Function for Retrieving Result from Vector DB:
After ingestion we can query the database directly and it will return us most close results according to semantic meaning.

This Function will return us top 5 results that matches with our query.

In [9]:
def retrieve_doc(query, chroma_collection):
    #we will pass the document path to doc_ingestion_in_vecdb() so that can ingest it in vector db after chunking and loading
    # chroma_collection= doc_ingestion_in_vecdb(path_to_doc)

    #lets get the 5 relevant results
    results = chroma_collection.query(query_texts=[query], n_results=5)

    #[0] means give the result of the first query, right now we have only 1 query
    retrieved_documents = results['documents'][0]

    return retrieved_documents

# Rag Function:

In [10]:
def rag(query, retrieved_documents, model, tokenizer):

    #we will join the retrieved documents (that are the relavant documents) into one variable
    messages = f"""
          You are a helpful expert of Machine Learning and Deep Learning.
          Your users are asking questions about information contained in an Machine Learning and Deep Learning Book."
          You will be shown the user's question, and the relevant information from the book.
          Answer the user's question using only this relevant information. Try to build a good answer uing this information.

          Give Precise answer according to User's Instruction, like if user wants explaination then provide explaination.
          If User want an answer in one or two lines then give answer accordingly.

        User's Question: {query}. \n Information from the book of Machine Learning and Deep Learning: {retrieved_documents}"
    """
    pipe = pipeline("text-generation",
                        model=model,
                        tokenizer=tokenizer)

    llm = HuggingFacePipeline(pipeline=pipe)
    # checking again that our model is working fine--->Asking LLM model the same question we asked our document
    content= llm(prompt=messages)

    return content


# Advance RAG Techniques:
## 1. Query Expansion Using the Hypothetical Answer from LLM:
So Basically instead of directly using our query to get the documents from vector database, firstly we will pass our query to LLM to get a hypothetical answer (that will help us to get answers like that from our document) from the LLM and then we will combine that hypothetical answer with our query to get an accurate and closely related answers/documents from the vector database.



In [11]:
def hypothetical_answer(query, model, tokenizer):
    messages = f"""
        You are a helpful expert Machine Learning and Deep Learning assistant.
        Provide an example answer to the given question,
        that might be found in a documents like from Book related to machine Learning and Deep Learning.

        The Question about which you have to give example answer is: {query}
    """
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer)

    llm = HuggingFacePipeline(pipeline=pipe)
    # checking again that our model is working fine--->Asking LLM model the same question we asked our document
    content= llm(prompt=query)

    #joining the original query and hypothetical answer
    joint_query = f"{query} {content}"

    return joint_query

In [12]:
original_query = "what is Linear Regression?"
joint_query = hypothetical_answer(original_query, model, tokenizer)
print(joint_query)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1290: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


what is Linear Regression? 
 everybody knows that Linear Regression is a statistical method used to establish a relationship between two or more variables. But do you know the history of Linear Regression?

The history of linear regression can be traced back to the early 19th century when Karl Friedrich Gauss, a German mathematician and astronomer, first introduced the concept of linear regression. Gauss observed that the relationship between the height of a person and their weight could be modeled using a linear equation, and he developed a method for estimating the parameters of such a model.

In the early 20th century, the concept of linear regression was further developed by other statisticians, including R.A. Fisher and S.S. Wilks. Fisher, in particular, made significant contributions to the field of linear regression, including the development of statistical methods for analyzing the results of experiments and the introduction of the concept of least squares estimation.

In the 1

**Now if you look above query, it has alot of hypothetical answer that can bring very accurate inforamtion about the query from the document, beacuase vector database will return answer similar to above query.**

### Making Vector DB Collection of our Doc:

In [13]:
path_to_doc= '/kaggle/input/aurelien-geron-hands-on-machine-learning/Aurelien-Geron-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow_-Concepts-Tools-and-Techniques-to-Build-Intelligent-Systems-OReilly-Media-2019.pdf'
chroma_collection= doc_ingestion_in_vecdb(path_to_doc)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

# Getting Relative Docs and Passing them to RAG:
We will get the relative documents using the joint query.

In [14]:
retrieved_documents= retrieve_doc(joint_query, chroma_collection)
retrieved_text= ""
for doc in retrieved_documents:
  retrieved_text+=doc

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**Now Lets get Answer from Our Doucments about this Query:**

In [15]:
retrieved_text

'them, and inverse them, and what partial derivatives are. if you are unfamiliar with these concepts, please go through the linear algebra and calculus introductory tutorials avail ‐ able as jupyter notebooks in the online supplemental material. for those who are truly allergic to mathematics, you should still go through this chapter and simply skip the equations ; hopefully, the text will be sufficient to help you understand most of the concepts. linear regression in chapter 1, we looked at a simple regression model of life satisfaction : life _ satisfac ‐ tion = θ0 + θ1 × gdp _ per _ capita. this model is just a linear function of the input feature gdp _ per _ capita. θ0 and θ1 are the model ’ s parameters. more generally, a linear model makes a prediction by simply computing a weighted1fun fact : this odd - sounding name is a statistics term introduced by francis galton while he was studying the fact that the children of tall people tend to be shorter than their parents. since child

In [16]:
#we will join the retrieved documents (that are the relavant documents) into one variable
messages = f"""
  You are a helpful expert of Machine Learning and Deep Learning.
  Your users are asking questions about information contained in an Machine Learning and Deep Learning Book."
  You will be shown the user's question, and the relevant information from the book.
  Answer the user's question using only this relevant information. Try to build a good answer uing this information.

  Give Precise answer according to User's Instruction, like if user wants explaination then provide explaination.
  If User want an answer in one or two lines then give answer accordingly.

User's Question: {original_query}. \n Information from the book of Machine Learning and Deep Learning: {retrieved_text}"
"""
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer)

llm = HuggingFacePipeline(pipeline=pipe)
# checking again that our model is working fine--->Asking LLM model the same question we asked our document
content= llm(prompt=messages)

In [17]:
print(content)


Please provide a clear and concise answer to the user's question based on the information provided in the book.

Answer: Linear regression is a type of machine learning algorithm that is used to model the relationship between a dependent variable (life satisfaction) and one or more independent variables (GDP per capita). In simple linear regression, the model makes a prediction by computing a weighted sum of the input features, where the weights are learned during the training process. The goal of linear regression is to minimize the distance between the model's predictions and the actual values of the dependent variable. In the book, the authors explain that linear regression is a linear function of the input features, and the parameters of the model (θ0 and θ1) are learned using a cost function that measures the distance between the model's predictions and the training examples.


## 2. Query Expansion Using the Related Queries:
Now in this technique basically we generate the queries related with our original queries and then we can use these queries to get more appropriate related documents from our vector database.


In [18]:
def generate_multiple_queries(query, model, tokenizer):
    messages = f"""
        You are a helpful expert Machine Learning and Deep Learning assistant.
        Tour Users are asking question related to machine Learning and Deep Learning.
        Suggest up to five additional related questions to help them find the information they need, for the provided question.
        Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic.
        Make sure they are complete questions, and that they are related to the original question.
        Output one question per line. Do not number the questions.

        The question about which you have to generate the question is {query}"""

    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer)

    llm = HuggingFacePipeline(pipeline=pipe)
    content= llm(prompt=messages)
    content = content.split("\n")
    return content

In [19]:
original_query = "What is deep learning?"
augmented_queries = generate_multiple_queries(original_query, model, tokenizer)

for query in augmented_queries:
    print(query)



        Example:
        User: What is deep learning?
        You: What are the different types of deep learning models?
        User: What is deep learning?
        You: How does deep learning work?
        User: What is deep learning?
        You: What are the applications of deep learning?

    }

    What are the different types of deep learning models?
    How does deep learning work?
    What are the applications of deep learning?
    What are the challenges of deep learning?
    How can I implement deep learning in my project?
}

Please let me know if you need any further assistance.


#### Combining Original and Generated Queries and Retrieving the Result:

In [20]:
queries = [original_query] + augmented_queries
retrieved_documents= retrieve_doc(queries, chroma_collection)
retrieved_text= ""
for doc in retrieved_documents:
  retrieved_text+=doc

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
retrieved_text

'part ii, neural networks and deep learning, covers the following topics : • what are neural nets? what are they good for? • building and training neural nets using tensorflow and keras. • the most important neural net architectures : feedforward neural nets, convolu ‐ tional nets, recurrent nets, long short - term memory ( lstm ) nets, autoencoders and generative adversarial networks ( gans ). • techniques for training deep neural nets. • scaling neural networks for large datasets. • learning strategies with reinforcement learning. • handling uncertainty with bayesian deep learning. the first part is based mostly on scikit - learn while the second part uses tensorflow and keras. don ’ t jump into deep waters too hastily : while deep learning is no doubt one of the most exciting areas in machine learning, you should master the fundamentals first. moreover, most problems can be solved quite well using simpler techniques such as random forests and ensemble methods ( discussed in part i )

#### Using Rag to Answer query from Retrieved Documents:
I wrote the below code again and again becuase somehow kaggle was not able to call rag() function that had this implementation :(

In [22]:
#we will join the retrieved documents (that are the relavant documents) into one variable
messages = f"""
  You are a helpful expert of Machine Learning and Deep Learning.
  Your users are asking questions about information contained in an Machine Learning and Deep Learning Book."
  You will be shown the user's question, and the relevant information from the book.
  Answer the user's question using only this relevant information. Try to build a good answer uing this information.

  Give Precise answer according to User's Instruction, like if user wants explaination then provide explaination.
  If User want an answer in one or two lines then give answer accordingly.

User's Question: {original_query}. \n Information from the book of Machine Learning and Deep Learning: {retrieved_text}"
"""
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer)

llm = HuggingFacePipeline(pipeline=pipe)
# checking again that our model is working fine--->Asking LLM model the same question we asked our document
content= llm(prompt=messages)

In [23]:
print(content)


Answer: Deep learning is a subfield of machine learning that focuses on complex neural networks with multiple layers. These networks are capable of learning and representing patterns in data, such as images, speech, or text. The term "deep learning" was coined to describe the use of these complex neural networks, which were previously known as artificial neural networks (ANNs). Deep learning is particularly useful for tasks that involve processing and analyzing large amounts of data, such as image recognition, speech recognition, or natural language processing.

In the book, the author covers various topics related to deep learning, including the different types of neural networks, such as feedforward neural nets, convolutive nets, recurrent nets, long short-term memory (LSTM) nets, autoencoders, and generative adversarial networks (GANs). The book also discusses techniques for training deep neural networks, scaling neural networks for large datasets, and handling uncertainty with Bay

**Result above looks Pretty Great :)**

## 2. Cross Encoder Re-Ranking:
Using this technique we can score the retrieval of the documents according to the query that we have set. Basically Every doccument will have a score, that will show how much related it is to our query.

**Sounds Interesting?, lets see what is inside:**

So basically main thing use in it is a Cross Encoder that is is present in the Sentence Transformer, When converting text to embeddings its process text together as single input, this allows model to directly compare and contrast the input, understanding their relation in the better way.

In Cross encoder we get score from 0 to 1, document having score close to 1 is most related query. We will give query and each retrieved document one by one to get the score of each document.

In [24]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#### Making Query-Doc Pairs for Comparison and Predicting the Score:

In [25]:
query= 'What is Neural Network?'
retrieved_documents= retrieve_doc(query, chroma_collection)
retrieved_text= ""
for doc in retrieved_documents:
  # retrieved_text+=doc
  print(doc)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

8in the 1990s, an ann with more than two hidden layers was considered deep. nowadays, it is common to see anns with dozens of layers, or even hundreds, so the definition of “ deep ” is quite fuzzy. 9 “ learning internal representations by error propagation, ” d. rumelhart, g. hinton, r. williams ( 1986 ). multi - layer perceptron and backpropagation an mlp is composed of one ( passthrough ) input layer, one or more layers of tlus, called hidden layers, and one final layer of tlus called the output layer ( see figure 10 - 7 ). the layers close to the input layer are usually called the lower layers, and the ones close to the outputs are usually called the upper layers. every layer except the output layer includes a bias neuron and is fully connected to the next layer. figure 10 - 7. multi - layer perceptron the signal flows only in one direction ( from the inputs to the out ‐ puts ), so this architecture is an example of a feedforward neural net ‐ work ( fnn ). when an ann contains a dee

In [26]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Scores:
-9.233899
3.365209
1.0348009
0.7302442
-6.1474347


**As you can see 4th result, 3rd, and 2nd results are matching more because they have high score**

Lets Check the Highest and Lowest scored results:

In [27]:
retrieved_documents[3]

'14 “ wide & deep learning for recommender systems, ” heng - tze cheng et al. ( 2016 ). as you can see, the sequential api is quite easy to use. however, although sequential models are extremely common, it is sometimes useful to build neural networks with more complex topologies, or with multiple inputs or outputs. for this purpose, keras offers the functional api. building complex models using the functional api one example of a non - sequential neural network is a wide & deep neural network. this neural network architecture was introduced in a 2016 paper by heng - tze cheng et al. 14. it connects all or part of the inputs directly to the output layer, as shown in figure 10 - 13. this architecture makes it possible for the neural network to learn both deep patterns ( using the deep path ) and simple rules ( through the short path ). in contrast, a regular mlp forces all the data to flow through the full stack of layers, thus simple patterns in the data may end up being distorted by th

In [28]:
retrieved_documents[0]

'8in the 1990s, an ann with more than two hidden layers was considered deep. nowadays, it is common to see anns with dozens of layers, or even hundreds, so the definition of “ deep ” is quite fuzzy. 9 “ learning internal representations by error propagation, ” d. rumelhart, g. hinton, r. williams ( 1986 ). multi - layer perceptron and backpropagation an mlp is composed of one ( passthrough ) input layer, one or more layers of tlus, called hidden layers, and one final layer of tlus called the output layer ( see figure 10 - 7 ). the layers close to the input layer are usually called the lower layers, and the ones close to the outputs are usually called the upper layers. every layer except the output layer includes a bias neuron and is fully connected to the next layer. figure 10 - 7. multi - layer perceptron the signal flows only in one direction ( from the inputs to the out ‐ puts ), so this architecture is an example of a feedforward neural net ‐ work ( fnn ). when an ann contains a de